In [1]:
# Created by Medad Rufus Newman on 19/03/2020
import pandas as pd
import requests
from pandas.io.json import json_normalize

In [42]:
# Download latest balloon data
balloon_ept_url = "http://habitat.habhub.org/habitat/_design/ept/_list/csv/payload_telemetry/flight_payload_time?include_docs=true&startkey=[%2299a403a8b8a8fbaa9de2332bcbf9605e%22,%221260d7d8adf6d27c5380a641ca7d786a%22]&endkey=[%2299a403a8b8a8fbaa9de2332bcbf9605e%22,%221260d7d8adf6d27c5380a641ca7d786a%22,[]]&fields=_sentence,_receivers,sentence_id,time,pressure_mbar,temperature_external,latitude,longitude,altitude,solar,satellites"
r = requests.get(balloon_ept_url, allow_redirects=True)
open('flight_payload_time.csv', 'wb').write(r.content)

176

In [3]:
# now read the data from habhub
import csv
with open('flight_payload_time.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    #for row in csv_reader:
        #print(row)

In [4]:
habub_df = pd.read_csv('flight_payload_time.csv')
habub_df.head()

,_sentence,_receivers,sentence_id,time,pressure_mbar,temperature_external,latitude,longitude,altitude,solar,satellites
0,"$$icspace19,3,09:12:08,545.9,-263.7,51.5098,-0...","eui-58a0cbfffe800c8a,eui-fcc23dfffe0a7ef5,eui-...",3,09:12:08,545.9,-263.7,51.5098,-0.118,0.0,2.44,0
1,"$$icspace19,3,09:12:09,545.9,-263.7,51.5098,-0...","eui-7276ff000b03239e,eui-7276ff000b03224d,eui-...",3,09:12:09,545.9,-263.7,51.5098,-0.118,0.0,2.44,0
2,"$$icspace19,3,09:12:10,545.9,-263.7,51.5098,-0...","eui-7276ff000b032627,eui-7276ff000b032390,eui-...",3,09:12:10,545.9,-263.7,51.5098,-0.118,0.0,2.44,0
3,"$$icspace19,4,09:58:53,545.9,-263.7,51.5098,-0...","eui-fcc23dfffe0b89c8,eui-b827ebfffe0e1b64,eui-...",4,09:58:53,545.9,-263.7,51.5098,-0.118,0.0,2.44,0
4,"$$icspace19,4,09:58:54,545.9,-263.7,51.5098,-0...","eui-7276ff000b03241e,eui-7276ff000b0325cd,eui-...",4,09:58:54,545.9,-263.7,51.5098,-0.118,0.0,2.44,0


In [5]:
habub_df['time'] = pd.to_datetime(habub_df['time'], format="%H:%M:%S")
habub_df.head()

,_sentence,_receivers,sentence_id,time,pressure_mbar,temperature_external,latitude,longitude,altitude,solar,satellites
0,"$$icspace19,3,09:12:08,545.9,-263.7,51.5098,-0...","eui-58a0cbfffe800c8a,eui-fcc23dfffe0a7ef5,eui-...",3,1900-01-01 09:12:08,545.9,-263.7,51.5098,-0.118,0.0,2.44,0
1,"$$icspace19,3,09:12:09,545.9,-263.7,51.5098,-0...","eui-7276ff000b03239e,eui-7276ff000b03224d,eui-...",3,1900-01-01 09:12:09,545.9,-263.7,51.5098,-0.118,0.0,2.44,0
2,"$$icspace19,3,09:12:10,545.9,-263.7,51.5098,-0...","eui-7276ff000b032627,eui-7276ff000b032390,eui-...",3,1900-01-01 09:12:10,545.9,-263.7,51.5098,-0.118,0.0,2.44,0
3,"$$icspace19,4,09:58:53,545.9,-263.7,51.5098,-0...","eui-fcc23dfffe0b89c8,eui-b827ebfffe0e1b64,eui-...",4,1900-01-01 09:58:53,545.9,-263.7,51.5098,-0.118,0.0,2.44,0
4,"$$icspace19,4,09:58:54,545.9,-263.7,51.5098,-0...","eui-7276ff000b03241e,eui-7276ff000b0325cd,eui-...",4,1900-01-01 09:58:54,545.9,-263.7,51.5098,-0.118,0.0,2.44,0


In [71]:
import datetime
all_transmissions  = []


class transmission:
    def __init__(self,unique_sentence_id,receivers,longitudes  = [],latitudes = []):
        self.unique_sentence_id = unique_sentence_id
        self.receivers = receivers
        self.longitudes = longitudes
        self.latitudes = latitudes
        
        


done_first_row = False
sentence_counter = 1
current_transmission_builder = None
working_time  = None

# WARNING: there appears to be one extra tx being counted. Need to test further

for index, row in habub_df.iterrows():
    
    if not done_first_row:
        working_time = row["time"]
        done_first_row = True
        
        first_tx  = transmission("transmission_"+str(sentence_counter),row["_receivers"].split(','))
        current_transmission_builder = first_tx
        all_transmissions.append(first_tx)
        sentence_counter +=1
        continue
    else:
        current_time = row["time"]
        delta_time = current_time - working_time
        working_time = current_time
        
        #print(delta_time)
        if delta_time< datetime.timedelta(seconds=5):
            # append to last tx object
            current_transmission_builder.receivers+=row["_receivers"].split(',')
        else:
            # make a new tx object
            current_tx  = transmission("transmission_"+str(sentence_counter),row["_receivers"].split(','))
            current_transmission_builder = current_tx
            all_transmissions.append(current_tx)
            sentence_counter +=1


In [72]:
# #all_transmissions
# #for i in all_transmissions:
#     print(i.unique_sentence_id,i.receivers,i.longitudes,i.latitudes)
#     print()

In [73]:
# read all gateway data
df_gateway_data  = pd.read_json("gateway-data.json")


In [74]:
df_gateway_data = df_gateway_data.T
#df_gateway_data.head()

In [75]:
# And in another column the df_gateway_data containing long and lat for each gateway
import numpy as np
latitudes = []
longitudes = []



for index, row in df_gateway_data.iterrows():
    if pd.notna(row["location"]):
        latitudes.append(row["location"]["latitude"])
        longitudes.append(row["location"]["longitude"])

    else:
        latitudes.append(np.nan)
        longitudes.append(np.nan)
        

df_gateway_data["latitude"] = latitudes
df_gateway_data["longitude"] = longitudes


In [76]:
#df_gateway_data.head()

In [77]:
# Now create the lookup table from our flight data

for i in all_transmissions:
    print(i)
    # Iterate through all the receivers and find their coordinates
    print("n receivers", len(i.receivers))
    for receiver in i.receivers:
        lookup_row  = df_gateway_data.loc[df_gateway_data['id'] == receiver]
        if lookup_row.empty:
            continue
        #print(receiver,lookup_row["id"].values,lookup_row["longitude"].values,lookup_row["latitude"].values)
        print(lookup_row["latitude"].values[0])
        i.latitudes.append(lookup_row["latitude"].values[0])
        i.longitudes.append(lookup_row["longitude"].values[0])
        print(i.latitudes)
        print("+++++++++++++++++++===++++++++++++++++++++++++++")

    print("=================================================================================================================")


n receivers 62
50.09430952
[50.09430952]
+++++++++++++++++++===++++++++++++++++++++++++++
48.77564342
[50.09430952, 48.77564342]
+++++++++++++++++++===++++++++++++++++++++++++++
48.38729249
[50.09430952, 48.77564342, 48.38729249]
+++++++++++++++++++===++++++++++++++++++++++++++
49.81729353
[50.09430952, 48.77564342, 48.38729249, 49.81729353]
+++++++++++++++++++===++++++++++++++++++++++++++
48.81964034
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
+++++++++++++++++++===++++++++++++++++++++++++++
48.83769179
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034, 48.83769179]
+++++++++++++++++++===++++++++++++++++++++++++++
49.7034777
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034, 48.83769179, 49.7034777]
+++++++++++++++++++===++++++++++++++++++++++++++
49.58967
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034, 48.83769179, 49.7034777, 49.58967]
+++++++++++++++++++===++++++++++++++++++++++++++
48.63114724
[50.09430952, 48.7

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [63]:
for i in all_transmissions:
    #print(i.unique_sentence_id,i.receivers,i.latitudes,i.longitudes)
    print(i.latitudes[:5])

[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]
[50.09430952, 48.77564342, 48.38729249, 49.81729353, 48.81964034]


In [209]:
i = all_transmissions[5]
df = pd.DataFrame(list(zip(i.receivers,i.latitudes,i.longitudes)), 
               columns =['receivers',"latitudes","longitudes"]) 
df 

,receivers,latitudes,longitudes
0,eui-7276ff000b03241d,50.094310,8.303840
1,nfckwthbl-lorapi,48.775643,11.415765
2,de-es-pv-2,48.387292,10.860886
3,eui-c0ee40ffff294723,49.817294,9.969320
4,eui-b827ebfffe3ac588,48.819640,9.161764
5,iten_home,48.837692,11.203765
6,cs7-gw1,49.703478,10.803503
7,eui-b827ebfffe3d6968,49.589670,8.320829
8,eui-b827ebfffea83863,48.631147,9.015215
9,tuebingen_wanne,49.335024,8.850312


In [210]:
# Now time to find the bounding circle around the coordinates for all transission.

# 
# Smallest enclosing circle - Library (Python)
# 
# Copyright (c) 2018 Project Nayuki
# https://www.nayuki.io/page/smallest-enclosing-circle
# 
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
# 
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
# 
# You should have received a copy of the GNU Lesser General Public License
# along with this program (see COPYING.txt and COPYING.LESSER.txt).
# If not, see <http://www.gnu.org/licenses/>.
# 

import math, random


# Data conventions: A point is a pair of floats (x, y). A circle is a triple of floats (center x, center y, radius).

# Returns the smallest circle that encloses all the given points. Runs in expected O(n) time, randomized.
# Input: A sequence of pairs of floats or ints, e.g. [(0,5), (3.1,-2.7)].
# Output: A triple of floats representing a circle.
# Note: If 0 points are given, None is returned. If 1 point is given, a circle of radius 0 is returned.
# 
# Initially: No boundary points known
def make_circle(points):
	# Convert to float and randomize order
	shuffled = [(float(x), float(y)) for (x, y) in points]
	random.shuffle(shuffled)
	
	# Progressively add points to circle or recompute circle
	c = None
	for (i, p) in enumerate(shuffled):
		if c is None or not is_in_circle(c, p):
			c = _make_circle_one_point(shuffled[ : i + 1], p)
	return c


# One boundary point known
def _make_circle_one_point(points, p):
	c = (p[0], p[1], 0.0)
	for (i, q) in enumerate(points):
		if not is_in_circle(c, q):
			if c[2] == 0.0:
				c = make_diameter(p, q)
			else:
				c = _make_circle_two_points(points[ : i + 1], p, q)
	return c


# Two boundary points known
def _make_circle_two_points(points, p, q):
	circ = make_diameter(p, q)
	left  = None
	right = None
	px, py = p
	qx, qy = q
	
	# For each point not in the two-point circle
	for r in points:
		if is_in_circle(circ, r):
			continue
		
		# Form a circumcircle and classify it on left or right side
		cross = _cross_product(px, py, qx, qy, r[0], r[1])
		c = make_circumcircle(p, q, r)
		if c is None:
			continue
		elif cross > 0.0 and (left is None or _cross_product(px, py, qx, qy, c[0], c[1]) > _cross_product(px, py, qx, qy, left[0], left[1])):
			left = c
		elif cross < 0.0 and (right is None or _cross_product(px, py, qx, qy, c[0], c[1]) < _cross_product(px, py, qx, qy, right[0], right[1])):
			right = c
	
	# Select which circle to return
	if left is None and right is None:
		return circ
	elif left is None:
		return right
	elif right is None:
		return left
	else:
		return left if (left[2] <= right[2]) else right


def make_diameter(a, b):
	cx = (a[0] + b[0]) / 2.0
	cy = (a[1] + b[1]) / 2.0
	r0 = math.hypot(cx - a[0], cy - a[1])
	r1 = math.hypot(cx - b[0], cy - b[1])
	return (cx, cy, max(r0, r1))


def make_circumcircle(a, b, c):
	# Mathematical algorithm from Wikipedia: Circumscribed circle
	ox = (min(a[0], b[0], c[0]) + max(a[0], b[0], c[0])) / 2.0
	oy = (min(a[1], b[1], c[1]) + max(a[1], b[1], c[1])) / 2.0
	ax = a[0] - ox;  ay = a[1] - oy
	bx = b[0] - ox;  by = b[1] - oy
	cx = c[0] - ox;  cy = c[1] - oy
	d = (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by)) * 2.0
	if d == 0.0:
		return None
	x = ox + ((ax*ax + ay*ay) * (by - cy) + (bx*bx + by*by) * (cy - ay) + (cx*cx + cy*cy) * (ay - by)) / d
	y = oy + ((ax*ax + ay*ay) * (cx - bx) + (bx*bx + by*by) * (ax - cx) + (cx*cx + cy*cy) * (bx - ax)) / d
	ra = math.hypot(x - a[0], y - a[1])
	rb = math.hypot(x - b[0], y - b[1])
	rc = math.hypot(x - c[0], y - c[1])
	return (x, y, max(ra, rb, rc))


_MULTIPLICATIVE_EPSILON = 1 + 1e-14

def is_in_circle(c, p):
	return c is not None and math.hypot(p[0] - c[0], p[1] - c[1]) <= c[2] * _MULTIPLICATIVE_EPSILON


# Returns twice the signed area of the triangle defined by (x0, y0), (x1, y1), (x2, y2).
def _cross_product(x0, y0, x1, y1, x2, y2):
	return (x1 - x0) * (y2 - y0) - (y1 - y0) * (x2 - x0)


In [211]:
i = all_transmissions[5]
print(len(list(zip(i.receivers,i.latitudes,i.longitudes))))

print(len(list(zip(i.latitudes,i.longitudes))))
#print(make_circle(zip(i.latitudes,i.longitudes)))

#print(make_circle(((1,1),(2,2),(3,3))))


85
689
